In [ ]:
import os
import json
import torch
import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"  #"rtzr/ko-gemma-2-9b-it" #"yanolja/EEVE-Korean-Instruct-10.8B-v1.0" 

device = 'cuda' if torch.cuda.is_available() else 'cpu'

llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True 
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm.eval()

In [ ]:
with open("../data/12000cleanlab+nannoise_8564.csv", "r") as f:
    df = pd.read_csv(f)
df = df[7000:]

In [ ]:
def augmentation(text: str):
    messages = [
    {"role": "system", 
    "content": "당신은 신문 기자입니다. 당신은 신문 기사 제목을 새롭게 만드는 임무를 부여받았습니다. 당신에게 신문 기사 컨텍스트 하나가 주어질 때 해당 컨텍스트와 비슷한 의미를 가지고 있는 신문 기사 스타일의 컨텍스트 한 개를 출력해야합니다. 이 때 모든 설명을 붙이지 말고 새로 만든 기사 제목 한 개만 출력하세요."},
    {"role": "user", "content": text}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )
    outputs = llm.generate(
        inputs.to(device),
        max_new_tokens=256,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.9,
        top_p=0.9,
    )

    # prompt_template = "Human: {prompt}\nAssistant:\n"
    # text2 = f"당신은 신문 기자입니다. 당신은 신문 기사 제목을 새롭게 만드는 임무를 부여받았습니다. 당신에게 신문 기사 컨텍스트 하나가 주어질 때 해당 컨텍스트와 비슷한 의미를 가지고 있는 신문 기사 스타일의 컨텍스트 한 개를 출력해야합니다. 이 때 모든 설명을 붙이지 말고 새로 만든 기사 제목 한 개만 출력하세요.{text}"
    # inputs = tokenizer(prompt_template.format(prompt=text2), return_tensors='pt')
    # outputs = llm.generate(
    #     **inputs.to(device),
    #     max_new_tokens=256,
    #     eos_token_id=tokenizer.eos_token_id,
    #     do_sample=True,
    #     temperature=0.1,
    #     top_p=0.9,
    # )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(generated_text)

    # result = generated_text.split("Assistant:")[-1].strip()
    result = generated_text.split("[|assistant|]")[-1].strip()
        
    # result = tokenizer.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True)
    
    return result

In [ ]:
from tqdm.notebook import tqdm 

augmented_data = []
for idx, row in tqdm(df.iterrows(), desc="Augmented", total=len(df)):
    text = row['text']
    print(f"########################### ORIGINAL TEXT {idx}  \"{text}\"")
    for i in range(1):
        result = augmentation(text)
        augmented_data.append({
            'ID': row['ID'],
            'text': result,
            'target': row['target']
        })
        print(result)


In [ ]:
augmented_doc = pd.DataFrame(augmented_data).sample(frac=1).reset_index(drop=True)
augmented_df = pd.concat([df, augmented_doc]).sample(frac=1).reset_index(drop=True)
augmented_df.to_csv("../data/12000cleanlab+nannoise_8564_truc_aug.csv", index=False, encoding="utf-8-sig")

In [ ]:
with open("../data/12000cleanlab+nannoise_8564_truc_aug.csv", "r") as f:
    df2 = pd.read_csv(f)

print(len(df2))